In [1]:
import pandas as pd
import numpy as np

# 1. **데이터 설명**
1. pathway_train, pathway_test, pathway_valid : CCLE에서 pathway score 계산 한 데이터
2. ic_train, ic_test, ic_val : 민감군, 저항군 라벨
3. ccle : 유전자 raw data -> 민감군, 저항군 간 유전자 발현량이 가장 큰 유전자 3개를 genes에 저장
4. tcga: 환자 데이터로 약물 민간/저항 라벨 부착 예정

In [3]:
pathway_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/IRINOTECAN/train_pathway_score_IRINOTECAN.csv")
pathway_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/IRINOTECAN/test_pathway_score_IRINOTECAN.csv")
pathway_valid = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/IRINOTECAN/val_pathway_score_IRINOTECAN.csv")

ic_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/IRINOTECAN/ic_train_IRINOTECAN.csv")
ic_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/IRINOTECAN/ic_test_IRINOTECAN.csv")
ic_val = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/IRINOTECAN/ic_val_IRINOTECAN.csv")

ccle = pd.read_csv("C:/Users/User/비어플 의료/#_filtered_CCLE_gene_expression.csv")
ccle.index = ccle["Unnamed: 0"]




In [4]:
pathway_train.set_index("SampleID", inplace= True)
pathway_test.set_index("SampleID", inplace= True)
pathway_valid.set_index("SampleID", inplace= True)

ic_train.set_index("Unnamed: 0", inplace=True)
ic_test.set_index("Unnamed: 0", inplace=True)
ic_val.set_index("Unnamed: 0", inplace=True)


In [5]:
genes = ["MAP4K1", "WAS", "IL16"  ]

# 2. **전처리**
1. pathway score scale
2. pathway score에 대해 PCA 진행 후 90% 설명력까지 사용
3. train 기준으로 fjt하고 trest, valid, TCGA에는 transform만 해주기
4. ccle, tcga 유전자들도 sacle해주기
5. ic라벨 인코딩`

In [7]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 스케일링 (z-score 정규화가 되어있더라도 PCA 전에 한 번 더 정규화)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pathway_train)
X_test_scaled = scaler.transform(pathway_test)
X_val_scaled = scaler.transform(pathway_valid)
# PCA 수행 (주성분 2개로 축소)
pca = PCA(n_components=0.90)
pathway_pca_train = pca.fit_transform(X_scaled)
pathway_pca_test  = pca.transform(X_test_scaled)
pathway_pca_valid = pca.transform(X_val_scaled)

In [8]:
pca_columns = [f"PC{i+1}" for i in range(pathway_pca_train.shape[1])]
pca_train_df_all = pd.DataFrame(pathway_pca_train, columns=pca_columns, index=pathway_train.index)
pca_train_df_all

pca_valid_df_all = pd.DataFrame(pathway_pca_valid, columns=pca_columns, index=pathway_valid.index)
pca_valid_df_all

pca_test_df_all = pd.DataFrame(pathway_pca_test, columns=pca_columns, index=pathway_test.index)
pca_test_df_all

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45
SampleID,,,,,,,,,,,,,,,,,,,,,
ACH-000678,-6.273242,0.140385,-3.042808,-0.913590,-2.087817,-2.987381,1.635341,1.087138,-2.692219,1.357856,...,1.177468,-0.972998,0.938035,-0.518741,-0.243983,0.841723,0.255365,-0.153097,0.147441,0.550964
ACH-000917,-13.270443,-3.460291,-2.621795,-1.857545,-2.073129,-3.313823,-1.297557,1.751297,1.224765,0.729729,...,0.697620,-1.546288,-0.560317,-1.288311,0.190643,0.257801,2.470549,2.541504,0.108214,1.659845
ACH-000856,-1.231217,-3.572928,1.546575,2.581406,-0.946056,2.196249,-0.236128,0.827925,0.005452,0.222713,...,-0.374271,-1.655216,-0.245429,0.022292,0.850046,0.358926,0.195050,-1.029833,-0.622277,-1.433993
ACH-000376,7.389518,4.650354,2.968333,1.851595,0.523655,1.137729,-0.139849,-3.311184,-1.569500,0.164978,...,-1.021293,0.179653,-0.746044,0.093102,0.563589,0.500899,-0.723359,-0.991982,0.647978,0.663661
ACH-000222,-7.934624,-2.103973,-6.250297,2.611709,-0.078040,-0.950124,0.765052,1.847289,-1.068089,1.725587,...,-0.659348,-1.020130,0.224527,1.548560,0.562981,-1.387213,0.594597,0.617647,0.267062,-1.615593
ACH-000805,3.937611,-0.081252,9.639593,-3.926861,2.390973,1.370180,1.751648,0.212189,0.084783,1.249767,...,-0.385763,1.257976,1.138771,1.208864,-0.191311,0.802314,-0.611845,-1.324088,0.264492,1.273772
ACH-000847,1.002788,-6.868649,6.179821,-1.163502,-1.622648,4.607000,-2.189540,-1.025686,1.473648,-0.321334,...,1.466566,0.234309,0.113391,-0.535362,-0.706168,0.871153,-0.499040,-0.075187,0.363010,0.109906
ACH-000832,-12.372387,1.003224,-7.044150,-1.613194,-0.605128,-2.252476,-0.687648,3.640919,0.611855,-1.542712,...,0.294098,-0.839146,1.636136,-0.736172,0.036923,0.349761,0.171052,2.315075,-0.721517,0.170360
ACH-000749,1.089267,-3.375322,9.222696,-0.195441,-1.243255,2.503964,-0.734464,-0.531727,1.474708,-0.246587,...,1.819980,-0.173285,0.247171,-1.254664,0.690805,1.049666,-0.422219,0.106334,1.214726,0.767913


In [9]:
ccle = ccle[genes]
ccle_expr_train = ccle.loc[pathway_train.index]
ccle_log_train = np.log2(ccle_expr_train+1)

scaler = StandardScaler()
ccle_scaled_train = pd.DataFrame(scaler.fit_transform(ccle_log_train),
                          columns = ccle_expr_train.columns,
                          index = ccle_expr_train.index)

ccle_expr_valid = ccle.loc[pathway_valid.index]
ccle_log_valid = np.log2(ccle_expr_valid+1)

ccle_scaled_valid = pd.DataFrame(scaler.transform(ccle_log_valid),
                          columns = ccle_expr_valid.columns,
                          index = ccle_expr_valid.index)

ccle_expr_test = ccle.loc[pathway_test.index]
ccle_log_test = np.log2(ccle_expr_test+1)
ccle_scaled_test = pd.DataFrame(scaler.transform(ccle_log_test),
                          columns = ccle_expr_test.columns,
                          index = ccle_expr_test.index)

In [10]:
X_train = pd.concat([pca_train_df_all, ccle_scaled_train], axis = 1)
X_valid = pd.concat([pca_valid_df_all, ccle_scaled_valid], axis = 1)
X_test = pd.concat([pca_test_df_all, ccle_scaled_test], axis = 1)

final_scaler = StandardScaler()
X_train_final = final_scaler.fit_transform(X_train)
X_valid_final = final_scaler.transform(X_valid)
X_test_final = final_scaler.transform(X_test)


In [11]:
# 숫자 라벨로 변환 (sensitive = 1, resistant = 0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(ic_train.iloc[:,0])
y_valid = le.fit_transform(ic_val.iloc[:,0])
y_test = le.fit_transform(ic_test.iloc[:,0])

# **3. 모델링**
1. 로지스틱, 랜덤포레스트, SVM, KNN으로 진행
2. 전처리를 R로 해야했기에 pipeline으로 묶을 수 없어 cross validation이 힘듬
3. 평가지표는 AUC, F1 score 중점으로 사용
4. 수동으로 튜닝 후 성능이 제일 좋은 모델 선택 후 TCGA 라벨 예측

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

In [32]:
## Logistic Regression
lr = LogisticRegression(C=0.001, max_iter=10000,class_weight="balanced")
lr.fit(X_train_final, y_train)
y_pred_lr = lr.predict(X_test_final)
y_proba_lr = lr.predict_proba(X_test_final)[:, 1]

## Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42,class_weight="balanced")
rf.fit(X_train_final, y_train)
y_pred_rf = rf.predict(X_test_final)
y_proba_rf = rf.predict_proba(X_test_final)[:, 1]

## Support Vector Machine (SVM)
svm = SVC(probability=True, random_state=42,class_weight="balanced")
svm.fit(X_train_final, y_train)
y_pred_svm = svm.predict(X_test_final)
y_proba_svm = svm.predict_proba(X_test_final)[:, 1]

## K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train_final, y_train)
y_pred_knn = knn.predict(X_test_final)
y_proba_knn = knn.predict_proba(X_test_final)[:, 1]

# ── AdaBoost 추가 ──
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(
    n_estimators=50,
    random_state=42
)
ada.fit(X_train_final, y_train)

y_pred_ada   = ada.predict(X_test_final)
y_proba_ada  = ada.predict_proba(X_test_final)[:, 1]

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [33]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

print("===== Validation Set Evaluation =====")

# Logistic Regression - Validation
y_pred_lr_valid = lr.predict(X_valid_final)
y_proba_lr_valid = lr.predict_proba(X_valid_final)[:, 1]
print("\n--- Logistic Regression ---")
print(classification_report(y_valid, y_pred_lr_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_lr_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_lr_valid))

# Random Forest - Validation
y_pred_rf_valid = rf.predict(X_valid_final)
y_proba_rf_valid = rf.predict_proba(X_valid_final)[:, 1]
print("\n--- Random Forest ---")
print(classification_report(y_valid, y_pred_rf_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_rf_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_rf_valid))

# Support Vector Machine - Validation
y_pred_svm_valid = svm.predict(X_valid_final)
y_proba_svm_valid = svm.predict_proba(X_valid_final)[:, 1]
print("\n--- Support Vector Machine ---")
print(classification_report(y_valid, y_pred_svm_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_svm_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_svm_valid))

# K-Nearest Neighbors - Validation
y_pred_knn_valid = knn.predict(X_valid_final)
y_proba_knn_valid = knn.predict_proba(X_valid_final)[:, 1]
print("\n--- K-Nearest Neighbors ---")
print(classification_report(y_valid, y_pred_knn_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_knn_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_knn_valid))

# AdaBoost - Validation
y_pred_ada_valid   = ada.predict(X_valid_final)
y_proba_ada_valid  = ada.predict_proba(X_valid_final)[:, 1]
print("\n--- AdaBoost ---")
print(classification_report(y_valid, y_pred_ada_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_ada_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_ada_valid))


===== Validation Set Evaluation =====

--- Logistic Regression ---
              precision    recall  f1-score   support

   resistant       0.54      0.54      0.54        24
   sensitive       0.59      0.59      0.59        27

    accuracy                           0.57        51
   macro avg       0.57      0.57      0.57        51
weighted avg       0.57      0.57      0.57        51

ROC-AUC: 0.5524691358024691
Confusion matrix:
 [[13 11]
 [11 16]]

--- Random Forest ---
              precision    recall  f1-score   support

   resistant       0.57      0.50      0.53        24
   sensitive       0.60      0.67      0.63        27

    accuracy                           0.59        51
   macro avg       0.59      0.58      0.58        51
weighted avg       0.59      0.59      0.59        51

ROC-AUC: 0.5632716049382716
Confusion matrix:
 [[12 12]
 [ 9 18]]

--- Support Vector Machine ---
              precision    recall  f1-score   support

   resistant       0.57      0.54    

In [36]:
from sklearn.ensemble import VotingClassifier
# ── Soft Voting Ensemble ──
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('svm', svm),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft'
)
voting_clf.fit(X_train_final, y_train)

y_pred_voting  = voting_clf.predict(X_test_final)
y_proba_voting = voting_clf.predict_proba(X_test_final)[:, 1]

print("\n===== Soft Voting Ensemble =====")
print(classification_report(y_test, y_pred_voting, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_voting))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_voting))





===== Soft Voting Ensemble =====
              precision    recall  f1-score   support

   resistant       0.55      0.46      0.50        24
   sensitive       0.58      0.67      0.62        27

    accuracy                           0.57        51
   macro avg       0.57      0.56      0.56        51
weighted avg       0.57      0.57      0.56        51

ROC-AUC: 0.5817901234567902
Confusion matrix:
 [[11 13]
 [ 9 18]]


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
